In [1]:
!pip install groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 5.3 MB/s eta 0:00:00


In [2]:
import os
import getpass
from groq import Groq

# Secure input (hidden)
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

client = Groq(api_key=os.environ["GROQ_API_KEY"])

print("Groq client initialized successfully!")

Enter your Groq API key: ··········
Groq client initialized successfully!


In [3]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": "You are a technical support expert. Be precise, analytical, and code-focused. Provide clear debugging steps and code examples when needed.",
        "temperature": 0.7
    },
    "billing": {
        "system_prompt": "You are a billing support expert. Be empathetic, professional, and policy-driven. Clearly explain refund or subscription policies.",
        "temperature": 0.7
    },
    "general": {
        "system_prompt": "You are a friendly and helpful assistant for general inquiries.",
        "temperature": 0.7
    },
    "tool": {
        "system_prompt": "You are a tool-use expert. If financial data is requested, delegate to the tool.",
        "temperature": 0.7
    }
}

In [11]:
def route_prompt(user_input):

    routing_prompt = f"""
    You are a strict intent classifier.

    Classify the following text into EXACTLY one of these categories:
    - technical (programming errors, debugging, software issues)
    - billing (payments, refunds, subscriptions, charges)
    - tool (real-time data like price of Bitcoin, stock prices, weather)
    - general (casual conversation or anything else)

    Return ONLY the category name.
    Do not explain.

    Text:
    {user_input}
    """

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": routing_prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip().lower()

In [12]:
def get_bitcoin_price():
    # Mock tool response
    return "The current price of Bitcoin is approximately $65,000 (mock data)."

In [13]:
def process_request(user_input):

    category = route_prompt(user_input)

    print(f"[Router Selected: {category}]")

    if category == "tool":
        return get_bitcoin_price()

    expert_config = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": expert_config["system_prompt"]},
            {"role": "user", "content": user_input}
        ],
        temperature=expert_config["temperature"]
    )

    return response.choices[0].message.content

In [14]:
print("Technical Test:")
print(process_request("My python script is throwing an IndexError on line 5."))

print("\nBilling Test:")
print(process_request("I was charged twice for my subscription this month."))

print("\nGeneral Test:")
print(process_request("Hello, how are you today?"))

print("\nTool Test:")
print(process_request("What is the current price of Bitcoin?"))

Technical Test:
[Router Selected: technical]
I'd be happy to help you debug your Python script. To provide a more accurate solution, I'll need some more information from you. Can you please provide the following:

1. The code snippet that's causing the error (lines 1-10 should be enough).
2. The full error message, including the line number (in this case, line 5).
3. Any relevant context or information about what the script is supposed to do.

A minimal, reproducible example (MRE) would be ideal. If you can't provide the actual code, try creating a simplified version that still reproduces the error.

When you provide this information, I'll do my best to:

* Identify the root cause of the error.
* Provide a clear explanation of what's going wrong.
* Offer a solution or debugging steps to fix the issue.

Please provide the necessary information, and I'll get started on helping you resolve the `IndexError` on line 5.

**Example of a minimal, reproducible example:**
```python
# example.py
